### Student Information
Name: 蕭靖澂 Ching-Cheng Hsiao

Student ID: 110033632

GitHub ID: hsiaooo

Kaggle name:

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2021-Lab2-master Repo](https://github.com/fhcalderon87/DM2021-Lab2-master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/c/dm2021-lab2-hw2/) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 24th 11:59 pm, Friday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 29th 11:59 pm, Wednesday)__. 

In [1]:
### Begin Assignment Here

### Load kaggle dataset

In [2]:
# load the libraries
import os
import json
import pandas as pd
import numpy as np

# load the json
path       = os.path.dirname(os.path.abspath("tweets_DM.json"))
# raw_data = pd.read_json(path+"/dataset/tweets_DM.json",lines=True)
# tweets   = pd.json_normalize(data=raw_data['_source'])
# identify = pd.read_csv(path+"/dataset/data_identification.csv")
# emotion  = pd.read_csv(path+"/dataset/emotion.csv")

In [3]:
# tweets.head()

In [4]:
# # rename column names
# tweets = tweets.rename(index=str, columns={"tweet.text":"text", "tweet.tweet_id":"tweet_id", "tweet.hashtags":"hashtags"})

# # add identify tags to dataframe
# tweets = pd.merge(tweets, identify, on="tweet_id")

# tweets.head()

In [5]:
# # get training set and test set
# train_df = tweets[tweets["identification"] == "train"]
# test_df  = tweets[tweets["identification"] == "test"]

# # drop identification tags
# train_df.drop(columns=["identification"], inplace=True)
# test_df.drop(columns=["identification"], inplace=True)

# print(f'Training set\n{train_df.head()}\n')
# print(f'Testing set\n{test_df.head()}')

In [6]:
# add emotion column
# train_df = pd.merge(train_df, emotion, on="tweet_id")
# test_df = test_df.assign(emotion="")

In [7]:
# use tweet_id as index
# train_df.set_index("tweet_id",inplace=True)
# test_df.set_index("tweet_id",inplace=True)

# train_df.head()

In [8]:
# save to pickle file
# train_df.to_pickle("train_df.pkl")
# test_df.to_pickle("test_df.pkl")

In [9]:
# load a pickle file
train_df = pd.read_pickle(path+"/train_df.pkl")
test_df  = pd.read_pickle(path+"/test_df.pkl")

### Pre-processing

#### Tf-idf

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
# from nltk.tokenize import word_tokenize
# from nltk.tokenize.stanford import StanfordTokenizer

tknzr = TweetTokenizer(preserve_case=False)
# tknzr = word_tokenize()
# tknzr = StanfordTokenizer()
tfidf = TfidfVectorizer(max_features=50000, stop_words='english', tokenizer=tknzr.tokenize)

# fitting
tfidf.fit(train_df['text'])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_features=50000, stop_words='english',
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f70b4087d00>>)

In [11]:
# transforming training sets
X_train = tfidf.transform(train_df['text'])
X_train.shape

(1455563, 50000)

In [12]:
# transforming testing sets
X_test = tfidf.transform(test_df['text'])
X_test.shape

(411972, 50000)

In [13]:
# set pointers
y_train = train_df['emotion']
y_test = test_df['emotion']

In [14]:
print(y_test[0:4])

tweet_id
0x28b412    
0x2de201    
0x218443    
0x2939d5    
Name: emotion, dtype: object


### Train model

#### GridsearchCV

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [16]:
from sklearn.model_selection import train_test_split

x = X_train
y = y_train
grid_X_train, grid_X_test, grid_y_train, grid_y_test=train_test_split(x, y, test_size=0.3)

In [17]:
# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
# logreg=LogisticRegression(solver='lbfgs', max_iter=100)
# logreg_cv=GridSearchCV(logreg,grid, cv=10, scoring="f1_micro")
# logreg_cv.fit(grid_X_train, grid_y_train)

# print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
# print("accuracy :", logreg_cv.best_score_)
# print('extimator: ', logreg_cv.best_estimator_)

After use gridsearchcv to find the parameters, the result is below.</br>
##### Logistic regression
tuned hpyerparameters :(best parameters)  {'C': 1000.0, 'penalty': 'l2'}</br>
accuracy : 0.5562384343537021</br>
extimator:  LogisticRegression(C=1000.0)

In [18]:
# logreg2 = LogisticRegression(C=1000, penalty="l2")
# logreg2.fit(grid_X_train,grid_y_train)
# print("score: ", logreg2.score(grid_X_test, grid_y_test))

##### Logistic regression
Validation score:  0.555297032763947

#### Logistic regression

In [19]:
# copy the X_train, X_test, y_train, y_test for Logistic regression
lr_X_train = X_train
lr_X_test  = X_test
lr_y_train = y_train
lr_y_test  = y_test

In [20]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=6, n_jobs=-1, max_iter=1000)
lr.fit(lr_X_train,lr_y_train)

LogisticRegression(C=6, max_iter=1000, n_jobs=-1)

In [21]:
pred_result_lr = lr.predict(lr_X_test)
pred_result_lr.shape

(411972,)

In [22]:
# save the result
test_df['emotion']=pred_result_lr
test_df.drop(columns=['hashtags','text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/lr_tfidf50000.csv')
print('Saved file to ', path, '/submission/lr_tfidf50000.csv')

Saved file to  /home/neaf-2070/code/hsiao/data_mining/DMLab2/DM2021-Lab2-HW2 /submission/lr_tfidf30000.csv


If I use C=1000, penalty="l2", submit to kaggle, the score only 0.45819, less than use C=6, score is 0.45963.

#### XGBoost Classifier

In [23]:
# load a pickle file
train_df = pd.read_pickle(path+"/train_df.pkl")
test_df = pd.read_pickle(path+"/test_df.pkl")

# copy the X_train, X_test, y_train, y_test for XGBoost
xgb_X_train = X_train
xgb_X_test  = X_test
xgb_y_train = y_train
xgb_y_test  = y_test

In [24]:
emotion = pd.Categorical(xgb_y_train.iloc[:], categories=xgb_y_train.iloc[:].unique(), ordered=True)

xgb_y_train.iloc[:] = emotion.codes
print(xgb_y_train)
print('anger \t\t= ', xgb_y_train[9])
print('anticipation \t= ', xgb_y_train[0])
print('disgust \t= ', xgb_y_train[18])
print('fear \t\t= ', xgb_y_train[2])
print('joy \t\t= ', xgb_y_train[3])
print('sadness \t= ', xgb_y_train[1])
print('surprise \t= ', xgb_y_train[19])
print('trust \t\t= ', xgb_y_train[15])

tweet_id
0x376b20    0
0x2d5350    1
0x1cd5b0    2
0x1d755c    3
0x2c91a8    0
           ..
0x321566    3
0x38959e    3
0x2cbca6    3
0x24faed    3
0x34be8c    3
Name: emotion, Length: 1455563, dtype: object
anger 		=  4
anticipation 	=  0
disgust 	=  6
fear 		=  2
joy 		=  3
sadness 	=  1
surprise 	=  7
trust 		=  5


In [ ]:
import xgboost as xgb

xgbc = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xgbc.fit(xgb_X_train, xgb_y_train)

In [ ]:
pred_result_xgbc = xgbc.predict(xgb_X_test)

pred_result_xgbc.shape
print(pred_result_xgbc)

In [ ]:
print(pred_result_xgbc[-10:])

In [ ]:
pred_result = []
for i, emo in enumerate(pred_result_xgbc):
    if emo == 0:
        pred_result.append('anticipation')
    elif emo == 1:
        pred_result.append('sadness')
    elif emo == 2:
        pred_result.append('fear')
    elif emo == 3:
        pred_result.append('joy')
    elif emo == 4:
        pred_result.append('anger')
    elif emo == 5:
        pred_result.append('trust')
    elif emo == 6:
        pred_result.append('disgust')
    elif emo == 7:
        pred_result.append('surprise')

In [ ]:
pred = np.array(pred_result)
print(pred[-10:])

In [ ]:
# save the result
test_df['emotion'] = pred
test_df.drop(columns = ['hashtags','text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns = ['emotion']
test_df.to_csv(path+'/submission/xgbc_tfidf.csv')
print('Saved file to ', path, '/submission/xgbc_tfidf.csv')

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# for a classificaiton problem, you need to provide both training & testing data
DT_X_train = X_train
DT_y_train = y_train
DT_X_test = X_test
DT_y_test = y_test

# take a look at data dimension 
print('X_train.shape: ', DT_X_train.shape)
print('y_train.shape: ', DT_y_train.shape)
print('X_test.shape: ', DT_X_test.shape)
print('y_test.shape: ', DT_y_test.shape)

In [ ]:
emotion = pd.Categorical(DT_y_train.iloc[:], categories=DT_y_train.iloc[:].unique(), ordered=True)

DT_y_train.iloc[:] = emotion.codes
DT_y_train = DT_y_train.astype('int')
print(DT_y_train)

In [ ]:
# build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

# training!
DT_model = DT_model.fit(DT_X_train, DT_y_train)

# predict!
y_test_pred = DT_model.predict(DT_X_test)

# so we get the pred result
y_test_pred[:10]

In [ ]:
# save the result
test_df['emotion']= y_test_pred
test_df.drop(columns=['hashtags','text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/DT_tfidf.csv')
print('Saved file to ', path, '/submission/DT_tfidf.csv')

### LSTM

In [ ]:
from keras.models import Model
from keras.utils import np_utils
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import rmsprop_v2
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

In [ ]:
# load a pickle file
train_df = pd.read_pickle(path+"/train_df.pkl")
test_df = pd.read_pickle(path+"/test_df.pkl")

# set pointers
y_train = train_df['emotion']
y_test = test_df['emotion']

In [ ]:
max_words = 20000
max_len = 300
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(train_df['text'])

In [ ]:
train_seq = tok.texts_to_sequences(train_df['text'])
test_seq = tok.texts_to_sequences(test_df['text'])

train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)

print(train_seq_mat.shape)
print(test_seq_mat.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
# y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

In [ ]:
# train_emotion = pd.Categorical(y_train.iloc[:], categories=y_train.iloc[:].unique(), ordered=True)
# test_emotion = pd.Categorical(y_test.iloc[:], categories=y_test.iloc[:].unique(), ordered=True)

# y_train.iloc[:] = train_emotion.codes
# y_test.iloc[:] = test_emotion.codes

# print(y_train)
# print(y_test)

In [ ]:
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

In [ ]:
inputs = Input(name='inputs',shape=[max_len])
## Embedding
layer = Embedding(max_words+1,128,input_length=max_len)(inputs)
layer = LSTM(128)(layer)
layer = Dense(128,activation="relu",name="FC1")(layer)
layer = Dropout(0.5)(layer)
layer = Dense(output_shape,activation="softmax",name="FC2")(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss="categorical_crossentropy",optimizer=rmsprop_v2.RMSprop(),metrics=["accuracy"])

In [ ]:
model_fit = model.fit(train_seq_mat, y_train,batch_size=128, epochs=3, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)] )

In [ ]:
# predict the result using our model
pred_result_lstm = label_decode(label_encoder, model.predict(test_seq_mat, batch_size=128))
pred_result_lstm[:5]

In [ ]:
# save the result
test_df['emotion']=pred_result_lstm
test_df.drop(columns=['hashtags','text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/keras_tfidf.csv')
print('Saved file to ', path, '/submission/keras_tfidf.csv')

#### BERT

In [3]:
import os

# select your GPU. Note that this should be set before you load tensorflow or pytorch.
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# To use multiple GPUs, combine all GPU ID with commas
# e.g. >>> os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,3'

In [4]:
# the model you want to use. Available models can be found here: https://huggingface.co/models
#MODEL_NAME = 'distilbert-base-uncased'
MODEL_NAME = 'bert-base-uncased'

In [5]:
from transformers import AutoTokenizer # For tokenization

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

#### Play with BERTTokenizer

<small><i>*You can safely skip this section if you're already familar with BERTTokenizer.</i></small>

Let's play with this tokenizer a little bit before we go on.

Using this tokenizer is pretty easy: just call this object, and it processes the sentences for you.  

In [6]:
example = "This so-called \"Perfect Evening\" was so disappointing, as well as discouraging us from coming to your Circle Theatre again."

embeddings = tokenizer(example)
embeddings

{'input_ids': [101, 2023, 2061, 1011, 2170, 1000, 3819, 3944, 1000, 2001, 2061, 15640, 1010, 2004, 2092, 2004, 12532, 4648, 4726, 2149, 2013, 2746, 2000, 2115, 4418, 3004, 2153, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}